In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from gee_scripts.get_sources import get_explanatory_composite
from gee_scripts.parameters import explain_vars
import ee
ee.Initialize()

# 1. Get explanatory composite

## 1.1. Set parameters

In [3]:
max_days_offset = 30
scale = 10
target_date = "2022-04-21"
output_image_name = "Test_area_2022-04-21"
ee_region = ee.Geometry.Polygon(
        [[[114.5409692136388, -2.456131528807272],
          [114.5409692136388, -2.5772075463747712],
          [114.72396024147083, -2.5772075463747712],
          [114.72396024147083, -2.456131528807272]]]);

In [4]:
# Get explanatory composite closest to target date
image = get_explanatory_composite(
    target_date=target_date, 
    ee_region=ee_region, 
    max_days_offset=max_days_offset
).select(explain_vars)


In [5]:
# Create a task and start it
task = ee.batch.Export.image.toDrive(
    image=image,
    description="test",
    folder="WorkshopIndonesia",
    fileNamePrefix=output_image_name,
    region=ee_region,
    scale=scale,
    maxPixels=1e13,
)

# Start the task
task.start()

# 2. Estimate GWL using the trained model

In [2]:
from pathlib import Path
import rasterio as rio
import numpy as np
import dask.array as da
import joblib
from sklearn.ensemble import RandomForestClassifier

In [3]:
explanatory_path = Path("data/11_explanatory_composites")
output_estimation_path = Path("data/12_estimated_gwl")
model_name = Path("data/10_models/Best_stations_Kalimantan_corr_0.3_temporal_variables.joblib")

# load the model with joblib
estimator = joblib.load(model_name)

In [4]:
input_image_name = Path("Test_area_2022-04-21.tif")

# Open the explanatory composite
with rio.open(explanatory_path/input_image_name) as src:
    img_array = src.read()
    meta = src.meta

transposed_img = np.transpose(img_array, (1, 2, 0))
reshaped_img = transposed_img.reshape(-1, 28)
predicted_values = estimator.predict(reshaped_img)

# Reshape the predicted values back to original shape
predicted_values = predicted_values.reshape(transposed_img.shape[:2])

/home/dguerrero/module-venv/gwl-modeling/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [6]:
# Reshape back to 2D grid
output_array = predicted_values.reshape((meta['height'], meta['width']))

# Update metadata for 1 band output
meta.update({'count': 1})

# Save to disk
with rio.open(output_estimation_path/f"{model_name.stem}_{input_image_name.stem}.tif", 'w', **meta) as dst:
    dst.write(output_array, 1)

# Predict over one station

In [34]:
import pandas as pd

In [38]:
df = pd.read_csv("data/9_clean_training_data/clean_training_data.csv")
len(df)

29768

In [43]:
# drop all stations with less tan 9 observations
min_obs = 9
df.groupby('id').agg({'date': 'count'}).sort_values(by='date', ascending=False).reset_index()
df = df.groupby('id').filter(lambda group: len(group) >= min_obs)
df

,source,id,date,gwl_cm,lat,lon,LIA,VH,VV,VVVH_ratio,...,B3,B4,B5,B7,ndvi,ndmi,ndbri,doy,region_id,percentile
28,pkeg,02_AHL_SBG-C027,2021-01-08,-38.0,3.916420,117.135840,34.944020,-12.068724,-8.078082,0.093560,...,23,69,63,30,0.500000,0.045455,0.393939,8,10,0.086651
29,pkeg,02_AHL_SBG-C027,2021-03-21,-38.0,3.916420,117.135840,34.947338,-11.666287,-7.412375,0.113317,...,23,69,63,30,0.500000,0.045455,0.393939,80,10,0.086651
30,pkeg,02_AHL_SBG-C027,2021-09-17,-41.0,3.916420,117.135840,34.941146,-14.317974,-8.376357,0.108333,...,23,69,63,30,0.500000,0.045455,0.393939,260,10,0.086651
31,pkeg,02_AHL_SBG-C027,2021-12-22,-38.0,3.916420,117.135840,34.945076,-19.224876,-9.127300,0.110302,...,23,69,63,30,0.500000,0.045455,0.393939,356,10,0.086651
32,pkeg,02_AHL_SBG-C027,2022-01-27,-38.0,3.916420,117.135840,34.943610,-13.070434,-6.776516,0.160750,...,39,58,104,60,0.195876,-0.283951,-0.016949,27,10,0.086651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29763,old_brg,kalteng1,2020-02-24,-23.9,-2.319728,114.058131,32.478706,-12.878587,-8.271499,0.097345,...,19,81,53,21,0.620000,0.208955,0.588235,55,8,0.994688
29764,old_brg,kalteng1,2020-03-07,-31.1,-2.319728,114.058131,32.478589,-13.078360,-6.593164,0.169898,...,19,81,53,21,0.620000,0.208955,0.588235,67,8,0.994688
29765,old_brg,kalteng1,2020-03-19,-14.7,-2.319728,114.058131,32.479968,-12.563061,-7.793232,0.110794,...,19,81,53,21,0.620000,0.208955,0.588235,79,8,0.994688
29766,old_brg,kalteng1,2020-03-31,-27.1,-2.319728,114.058131,32.480674,-13.179518,-7.523130,0.128794,...,19,81,53,21,0.620000,0.208955,0.588235,91,8,0.994688
